# 4. A Practical Example of Wolfram Cloud Integration

Below is an example of how to utilise functionality specific to the Wolfram Language in a workflow.

## Access the Wolfram Knowledgebase to augment data 

First, load a Pandas dataframe:

In [ ]:
import pandas as pd
import numpy as np

from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession() # May need to provide a file path. See https://wolfr.am/177vH2umJ

 Get world GDP data and look at the first 5 rows:

In [ ]:
import os
path = os.path.join(os.getcwd(), 'AdditionalFiles')
gdp = pd.read_csv(os.path.join(path, 'WorldBankGDP.csv'), skiprows=4)
gdp.head(5)

Clean up the data:

In [ ]:
gdp = gdp.drop(['Indicator Name', 'Indicator Code', 'Country Code', 'Unnamed: 66'], axis=1) # Drop unnecessary columns
gdp = gdp[gdp['Country Name'] != 'World'] # Remove 'World' from the country list

Reduce the size of the data for this demonstration:

In [ ]:
gdp = gdp.head(5)
gdp

Define a WL function which takes a country name and returns a (dated) population:

In [ ]:
session.evaluate(wlexpr('''
ClearAll[pop];
pop[country_String, year_ : DateString["Year"]] := Block[
  {countryEntity, population},
  countryEntity = Interpreter["Country"][country];
  population = EntityValue[countryEntity, Dated["Population", ToExpression[year]]];
  If[
    Or[
      FailureQ[countryEntity], UnsameQ[Head[population], Quantity]
    ],
    Indeterminate,
    QuantityMagnitude[population],
    Indeterminate
  ]
]
'''))

population = session.function(wlexpr('pop'))

Check that the function is working as expected:

In [ ]:
print(population('UK', 2010))
print(population('NotACountry', 2010))

Apply the function to each row of the dataset, replacing all of the GDPs with populations. Due to individual Interpreter and Entity calls – each of which has some overhead – this may take a little while:

In [ ]:
for year in range(1960, 2021):
    gdp[str(year)] = gdp.apply(lambda row: population(row['Country Name'], str(year)), axis=1)
gdp

See how the population of each country has grown over a 50-year period (1960 - 2010):

In [ ]:
gdp[['Country Name', '1960', '2010']]

Plot the population of Aruba over time, using a mixture of Python and Wolfram Language:

In [ ]:
dataClean = wl.Association(wl.Query(wl.All, session.evaluate(wlexpr('First[#] -> KeyValueMap[{ToExpression[#1], #2}&, Rest[#]]&')))(wl.Normal(wl.Values(gdp))))

session.evaluate(wlexpr('''
plotterWL[data_] := With[
    {country = "Aruba"},
    ListLinePlot[
        data[country],
        PlotLabel -> country,
        Frame -> True,
        FrameLabel -> {"Year", "Population"}
    ]
]
'''))
plotter = session.function('plotterWL')

graphic = plotter(dataClean)
path = os.path.join(os.getcwd(), 'AdditionalFiles')
png_export = wl.Export(os.path.join(path, 'PopulationPlot.png'), graphic, "PNG")
session.evaluate(png_export)

from PIL import Image
Image.open(os.path.join(path, 'PopulationPlot.png'))